# 🚀 Customize `gpt-oss` model using SageMaker HyperPod recipes and HyperPod on EKS

---
In this notebook, we use [SageMaker HyperPod recipes](https://github.com/aws/sagemaker-hyperpod-recipes) to fine-tune the GPT-OSS models. Recipes support fine-tuning the following latest released GPT-OSS models,
* [openai/gpt-oss-20b](https://huggingface.co/openai/gpt-oss-20b)
* [openai/gpt-oss-120b](https://huggingface.co/openai/gpt-oss-120b)

In this notebook, we show you how to use the recipes with SageMaker HyperPod on EKS. To run recipes on SageMaker training jobs, see [finetune_gpt_oss_hyperpod_recipes_tj.ipynb](https://github.com/aws-samples/amazon-sagemaker-generativeai/blob/main/3_distributed_training/models/openai--gpt-oss/finetune_gpt_oss_hyperpod_recipes_tj.ipynb)

**What are GPT‑OSS Models?**

OpenAI released **gpt‑oss‑120b** and **gpt‑oss‑20b** on **August 5, 2025**—its first open‑weight language models since GPT‑2. These models are provided under the **Apache 2.0 license**, enabling both commercial and non-commercial use with full access to the model weights.

- **gpt‑oss‑120b**  
  - ~117 billion parameters, but only ~5.1 billion active per token via Mixture‑of‑Experts (MoE) routing  
  - 36 layers, 128 experts total, with 4 active per token  
  - Supports up to **128 k context length** using dense + sparse attention, grouped multi‑query attention, and RoPE

- **gpt‑oss‑20b**  
  - ~21 billion parameters, ~3.6 billion active per token  
  - 24 layers, 32 total experts, with 4 active per token  
  - Same efficient attention and context‑length capabilities as the large variant 

These models support **chain‑of‑thought (CoT) reasoning**, structured outputs, and are compatible with the OpenAI Responses API. You can adjust reasoning effort (low/medium/high) with a simple system message—balancing latency against performance.

- **gpt‑oss‑120b** matches or exceeds the performance of OpenAI’s proprietary **o4‑mini** model on benchmarks such as Codeforces (coding), MMLU and HLE (general reasoning), HealthBench (health), and AIME (competition math).
- **gpt‑oss‑20b**, despite its smaller size, outperforms **o3‑mini** across similar benchmarks, especially in mathematics and coding domains.

---

## SageMaker HyperPod on EKS
This notebook assumes you already have a HyperPod cluster orchestrated by Amazon EKS setup, and you have `ml.p5.48xlarge` instances in your cluster. If you don't have it setup, follow the instructions to setup a cluster:

1. Request the following SageMaker quotas on the Service Quotas console:

    `P5 instances (ml.p5.48xlarge) for HyperPod clusters (ml.p5.48xlarge for cluster usage): 1`

2. Set up a HyperPod EKS cluster, referring to [Amazon SageMaker HyperPod Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-hyperpod-eks.html). You can use the [New console experience](https://catalog.workshops.aws/sagemaker-hyperpod-eks/en-US/00-setup/new-console-experience) to create the cluster, or alternatively you can also use the  CloudFormation template provided in the [HyperPod EKS workshop](https://catalog.workshops.aws/sagemaker-hyperpod-eks/en-US/00-setup/00-workshop-infra-cfn) and follow the instructions to [set up a cluster](https://catalog.workshops.aws/sagemaker-hyperpod-eks/en-US/01-cluster) and a development environment to access and submit jobs to the cluster.

3. Setup an Amazon FSx for Lustre file system for saving and loading data/checkpoints. You can follow the instructions under [Set Up an FSx for Lustre File System](https://catalog.workshops.aws/sagemaker-hyperpod-eks/en-US/01-cluster/06-fsx-for-lustre) to setup a FSx Lustre volume and associate it with the cluster.

Run the remainder of the notebook in an environment that has access to the cluster. For options, see [Set up your environment](https://catalog.workshops.aws/sagemaker-hyperpod-eks/en-US/00-setup/env-setup).



## Prepare your data

This section assumes you are running this from an environment that has the `/fsx` volume mounted, using the instructions above. We will pre-process the dataset and save it to the FSx volume that's mounted to all pods.

In [ ]:
from datasets import load_dataset
 
from transformers import AutoTokenizer
import numpy as np
 
dataset = load_dataset("HuggingFaceH4/Multilingual-Thinking", split="train")
 
tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-120b")
messages = dataset[0]["messages"]
conversation = tokenizer.apply_chat_template(messages, tokenize=False)
print(conversation)
 
def preprocess_function(example):
    return tokenizer.apply_chat_template(example['messages'], 
                                        return_dict=True, 
                                        padding="max_length", 
                                        max_length=4096, 
                                        truncation=True)
 
def label(x):
    x["labels"]=np.array(x["input_ids"])
    x["labels"][x["labels"]==tokenizer.pad_token_id]=-100
    x["labels"]=x["labels"].tolist()
    return x
 
dataset = dataset.map(preprocess_function, 
                      remove_columns=['reasoning_language', 
                                      'developer', 
                                      'user', 
                                      'analysis', 
                                      'final',
                                      'messages'])
dataset = dataset.map(label)

# for HyperPod, save to mounted FSx volume
dataset.save_to_disk("/fsx/multilingual_4096")

## Set up your environment

To fine-tune using HyperPod recipes, start by setting up the virtual environment and installing all necessary dependencies to run the job on the EKS cluster.

In [ ]:
# create a virtual environment
!python3 -m venv ${PWD}/venv
!source venv/bin/activate

In [ ]:
# download and set up the HyperPod recipes repo
!git clone --recursive https://github.com/aws/sagemaker-hyperpod-recipes.git
!cd sagemaker-hyperpod-recipes
!pip3 install -r requirements.txt 

You can now use the recipes' `launch_scripts` to submit your training job! Follow the steps below to submit the job:

1. In `recipes_collection/cluster/k8s.yaml`, update `persistent_volume_claims` section. It mounts the FSx claim to the `/fsx` directory of each computing pod.
```
- claimName: fsx-claim    
  mountPath: fsx
```
2. Update the launch script for the GPT-OSS 120B model, available in `launcher_scripts/gpt_oss/run_hf_gpt_oss_120b_seq4k_gpu_lora.sh`, with the `cluster_type` of your HyperPod cluster. In out case, we will update the script with `cluster=k8s` and `cluster_type=k8s` values to the script. Your modified script should look like below:
```
#!/bin/bash

# Original Copyright (c), NVIDIA CORPORATION. Modifications © Amazon.com

#Users should setup their cluster type in /recipes_collection/config.yaml

SAGEMAKER_TRAINING_LAUNCHER_DIR=${SAGEMAKER_TRAINING_LAUNCHER_DIR:-"$(pwd)"}

HF_MODEL_NAME_OR_PATH="openai/gpt-oss-120b" # HuggingFace pretrained model name or path

TRAIN_DIR="/fsx/multilingual_4096" # Location of training dataset
VAL_DIR="/fsx/multilingual_4096" # Location of validation dataset

EXP_DIR="/fsx/experiment" # Location to save experiment info including logging, checkpoints, ect
HF_ACCESS_TOKEN="hf_xxxxxxxx" # Optional HuggingFace access token

HYDRA_FULL_ERROR=1 python3 "${SAGEMAKER_TRAINING_LAUNCHER_DIR}/main.py" \
    recipes=fine-tuning/gpt_oss/hf_gpt_oss_120b_seq4k_gpu_lora \
    container="658645717510.dkr.ecr.us-west-2.amazonaws.com/smdistributed-modelparallel:sm-pytorch_gpt_oss_patch_pt-2.7_cuda12.8" \
    base_results_dir="${SAGEMAKER_TRAINING_LAUNCHER_DIR}/results" \
    recipes.run.name="hf-gpt-oss-120b-lora" \
    cluster=k8s \ # Imp: add cluster line when running on HP EKS
    cluster_type=k8s \ # Imp: add cluster_type line when running on HP EKS
    recipes.exp_manager.exp_dir="$EXP_DIR" \
    recipes.trainer.num_nodes=1 \
    recipes.model.data.train_dir="$TRAIN_DIR" \
    recipes.model.data.val_dir="$VAL_DIR" \
    recipes.model.hf_model_name_or_path="$HF_MODEL_NAME_OR_PATH" \
    recipes.model.hf_access_token="$HF_ACCESS_TOKEN" \
```

3. Now, launch the job by running the launcher script:
```
chmod +x launcher_scripts/gpt_oss/run_hf_gpt_oss_120b_seq4k_gpu_lora.sh 
bash launcher_scripts/gpt_oss/run_hf_gpt_oss_120b_seq4k_gpu_lora.sh
```

## Monitoring the job

You can now use `kubectl` commands to monitor your job. Run `kubectl get pods` to see the pod running your job.


Once you have the pod name, you can use the `logs` command as shown below to monitor the job:
```
kubectl logs -f hf-gpt-oss-120b-lora-h2cwd-worker-0
```

When the status of the pod gets to `Completed`, the final merged model can be found in the experiment directory path we defined in the launcher script under `/fsx/experiment/checkpoints/peft_full/steps_50/final-model`.

That's it! Your fine-tuning using HyperPod recipes is now complete! To deploy the model for inference, follow the steps in [finetune_gpt_oss.ipynb](https://github.com/aws-samples/amazon-sagemaker-generativeai/blob/main/3_distributed_training/models/openai--gpt-oss/finetune_gpt_oss.ipynb).